In [133]:
from re import L
import networkx as nx
import csv

class Graph:
    def __init__(self):
        # Initialize an empty NetworkX graph
        self.G = nx.Graph()
        self.stream_paths = {}  # Dictionary to store paths for each stream
        # Dictionary to store queue assignments for each output port
        self.queue_assignments = {}

    def load_from_csv(self, topology_file):
        """
        Loads a topology from a CSV file and populates the graph with nodes and edges.

        Args:
            topology_file (str): Path to the topology CSV file.
        """
        devices = []
        links = []

        # Use the csv module to read the file
        with open(topology_file, 'r') as file:
            reader = csv.reader(file)
            for fields in reader:
                if fields[0].lower() == 'es' or fields[0].lower() == 'sw':  # Device entry (SW or ES)
                    devices.append(fields)
                elif fields[0].lower() == 'link':  # Link entry
                    links.append(fields)

        # Add nodes (SW: Switch, ES: End System)
        for device in devices:
            device_type = device[0].strip()
            device_name = device[1].strip()
            self.G.add_node(device_name, type=device_type)

        # Add edges (LINK)
        for link in links:
            link_id = link[1].strip()
            source_device = link[2].strip()
            source_port = int(link[3].strip())
            destination_device = link[4].strip()
            destination_port = int(link[5].strip())

            # Add the edge (link) between source and destination devices
            self.G.add_edge(source_device, destination_device, link_id=link_id,
                            source_port=source_port, destination_port=destination_port)

    def load_streams(self, streams_file):
        """
        Loads stream information from a CSV file.

        Args:
            streams_file (str): Path to the streams CSV file.
        """
        streams = []
        with open(streams_file, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                streams.append({
                    'pcp': int(row[0]),
                    'name': row[1].strip(),
                    'type': row[2].strip(),
                    'source': row[3].strip(),
                    'destination': row[4].strip(),
                    'size': int(row[5]),
                    'period': float(row[6]),
                    'deadline': float(row[7])
                })
        self.streams = streams
        
        # debug print
        # for stream in self.streams:
        #     print(stream)


    def find_shortest_path(self, node_a, node_b):
        """
        Finds the shortest path between two nodes using Dijkstra's algorithm.

        Args:
            node_a (str): The source node.
            node_b (str): The destination node.

        Returns:
            path (list): The shortest path from node_a to node_b as a list of nodes.
        """
        try:
            # Verify that nodes exist in the graph
            if node_a not in self.G or node_b not in self.G:
                print(
                    f"One or both of the nodes ({node_a}, {node_b}) are not in the graph.")
                return None

            # Use Dijkstra's algorithm to find the shortest path without weights
            path = nx.shortest_path(self.G, source=node_a, target=node_b)
            return path
        except nx.NetworkXNoPath:
            print(f"No path exists between {node_a} and {node_b}.")
            return None
        except nx.NodeNotFound:
            print(
                f"One or both of the nodes ({node_a}, {node_b}) are not in the graph.")
            return None

    def calculate_all_paths(self):
        """
        Calculates and stores the shortest paths for all streams.
        """
        for stream in self.streams:
            source = stream['source']
            destination = stream['destination']
            path = self.find_shortest_path(source, destination)
            if path:
                annotated_path = []
                for i in range(len(path) - 1):
                    current_node = path[i]
                    link_id = self.G.get_edge_data(
                        current_node, path[i + 1])['link_id']
                    previous_node = path[i - 1] if i > 0 else 'N/A'
                    edge_data = self.G.get_edge_data(current_node, path[i + 1])
                    source_port = edge_data['source_port']
                    destination_port = edge_data['destination_port']
                    annotated_path.append(
                        f"{current_node}:{link_id}:{source_port}")
                annotated_path.append(
                    f"{path[-1]}:{destination_port}")
                self.stream_paths[stream['name']] = '->'.join(annotated_path)
                print(f"Stream {stream['name']} from {source} to {destination}: Annotated Path: {self.stream_paths[stream['name']]}")
            else:
                print(
                    f"No path found for Stream {stream['name']} from {source} to {destination}")

    def assign_queues(self):
        """
        Assigns shaped queues for each combination of priority level and upstream source for every output port.
        """
        for stream_name, path in self.stream_paths.items():
            stream = next(s for s in self.streams if s['name'] == stream_name)
            pcp = stream['pcp']
            path_nodes = path.split('->')
            for i in range(len(path_nodes) - 1):
                current_node = path_nodes[i].split(':')[0]
                previous_node = path_nodes[i -
                                           1].split(':')[0] if i > 0 else 'N/A'
                edge_data = self.G.get_edge_data(
                    current_node, path_nodes[i + 1].split(':')[0])
                output_port = edge_data['destination_port']
                key = (current_node, previous_node, output_port, pcp)

                if key not in self.queue_assignments:
                    self.queue_assignments[key] = []
                self.queue_assignments[key].append(stream_name)

        print("Queue Assignments:")

    def compute_worst_case_delay(self, stream_name):
        """
        Computes the worst-case per-hop delay for a stream over its path.

        Args:
            stream_name (str): The name of the stream.

        Returns:
            float: The computed worst-case delay for the stream.
        """
        if stream_name not in self.stream_paths:
            print(f"No path found for stream {stream_name}.")
            return None

        header_size = 42 # Ethernet frame header size 
        path = self.stream_paths[stream_name].split('->')
        stream = next(s for s in self.streams if s['name'] == stream_name)
        b = stream['size'] + header_size   # Include Ethernet frame overhead
        r = b / stream['period']
        l_min = b  # Minimum frame size
        r_link = 1e9  # Assume 1 Gbps link rate for now
        total_delay = 0
        print(f"\nCalculating delay for stream: {stream_name}")

        for i in range(len(path) - 1):
            current_node = path[i].split(':')[0]
            next_node = path[i + 1].split(':')[0]
            edge_data = self.G.get_edge_data(current_node, next_node)
            output_port = edge_data['destination_port']
            key = (current_node, path[i - 1].split(':')[0] if i > 0 else 'N/A', output_port, stream['pcp'])

            # Gather all streams at this output port across priority levels
            all_interfering_streams = [s_name for k, streams in self.queue_assignments.items() 
                                    if k[0] == current_node and k[2] == output_port for s_name in streams]

            # Organize interfering streams by priority level relative to `stream['pcp']`
            higher_priority_streams = []
            same_priority_streams = []
            lower_priority_streams = []

            for s_name in all_interfering_streams:
                s_data = next((s for s in self.streams if s['name'] == s_name), None)
                if s_data:
                    if s_data['pcp'] > stream['pcp']:
                        higher_priority_streams.append(s_name)
                    elif s_data['pcp'] == stream['pcp']:
                        same_priority_streams.append(s_name)
                    else:
                        lower_priority_streams.append(s_name)

            # Debug output for all interfering streams organized by priority
            print(f"  At hop {current_node} -> {next_node}, output port {output_port}")
            print("    Higher-priority streams:", higher_priority_streams)
            print("    Same-priority streams:", same_priority_streams)
            print("    Lower-priority streams:", lower_priority_streams)

            # Calculate r_H: Total rate of higher-priority streams
            r_H = sum(
                (s_data['size'] + header_size) / s_data['period']
                for s_name in higher_priority_streams
                for s_data in self.streams if s_data['name'] == s_name
            )
            
            # Calculate b_H once per hop as the total burst size of all higher-priority streams
            b_H = sum(
                (s_data['size'] + header_size) for s_name in higher_priority_streams
                for s_data in self.streams if s_data['name'] == s_name
            )
            
            # Calculate l_L once per hop as the maximum size of lower-priority streams
            l_L = max(
                [(s_data['size'] + header_size) for s_name in lower_priority_streams
                for s_data in self.streams if s_data['name'] == s_name] or [0]
            )
            
            # Debug output for r_H, b_H, and l_L
            print(f"    Calculated r_H = {r_H}, b_H = {b_H}, l_L = {l_L}")
            
            # Calculate b_C_j as the maximum burst of any same-priority interfering stream
            b_C_j = max(
                [(s_data['size'] + header_size) for s_name in same_priority_streams
                if s_name != stream_name 
                for s_data in self.streams if s_data['name'] == s_name] or [0]
            )
            print(f"    b_C_j (max same-priority burst) = {b_C_j}")

            # Calculate the per-hop delay using the worst-case conditions at this hop
            try:
                per_hop_delay = (b_H + b_C_j + (b - l_min) + l_L) / (r_link - r_H) + (l_min / r_link)
            except ZeroDivisionError:
                print(f"ZeroDivisionError: (r_link - r_H) resulted in zero for stream {stream_name} at hop {current_node}.")
                return None

            print(f"    Per-hop delay at {current_node} -> {next_node} for stream {stream_name}: {per_hop_delay}")
            total_delay += per_hop_delay

        print(f"Total delay for stream {stream_name}: {total_delay}")
        return total_delay




# Example usage
topology_file = 'small-topology.csv'  # Replace with your actual file path
streams_file = 'small-streams.csv'  # Replace with your actual file path
graph = Graph()
graph.load_from_csv(topology_file)
graph.load_streams(streams_file)

# Calculate all paths for the streams
graph.calculate_all_paths()

# Assign queues based on the Queue Assignment Rules
graph.assign_queues()

# Compute and print the worst-case delay for each stream
flows = ['Flow_0', 'Flow_1', 'Flow_10', 'Flow_11', 'Flow_12', 'Flow_13', 'Flow_14', 'Flow_6', 'Flow_7', 'Flow_8', 'Flow_9']
for name in flows:
    wcd = graph.compute_worst_case_delay(name)

Stream Flow_0 from node0_0_0_0 to node0_0_6_0: Annotated Path: node0_0_0_0:e3:2->sw_0_0:e1:0->sw_0_3:e6:2->sw_0_6:e21:3->node0_0_6_0:1
Stream Flow_1 from node0_0_4_1 to node0_0_3_1: Annotated Path: node0_0_4_1:e20:4->sw_0_4:e17:1->sw_0_6:e6:2->sw_0_3:e9:5->node0_0_3_1:1
Stream Flow_10 from node0_0_2_1 to node0_0_6_1: Annotated Path: node0_0_2_1:e24:3->sw_0_2:e5:1->sw_0_3:e6:2->sw_0_6:e22:4->node0_0_6_1:1
Stream Flow_11 from node0_0_5_1 to node0_0_6_0: Annotated Path: node0_0_5_1:e12:3->sw_0_5:e2:1->sw_0_0:e1:0->sw_0_3:e6:2->sw_0_6:e21:3->node0_0_6_0:1
Stream Flow_12 from node0_0_0_0 to node0_0_6_1: Annotated Path: node0_0_0_0:e3:2->sw_0_0:e1:0->sw_0_3:e6:2->sw_0_6:e22:4->node0_0_6_1:1
Stream Flow_13 from node0_0_4_1 to node0_0_6_0: Annotated Path: node0_0_4_1:e20:4->sw_0_4:e17:1->sw_0_6:e21:3->node0_0_6_0:1
Stream Flow_14 from node0_0_4_1 to node0_0_3_0: Annotated Path: node0_0_4_1:e20:4->sw_0_4:e17:1->sw_0_6:e6:2->sw_0_3:e8:4->node0_0_3_0:1
Stream Flow_15 from node0_0_4_1 to node0_0_1